# This code measures how fast it can stream full events from Elasticsearch

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from time import time

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

## Select data format you would like to stream
options are data15_13tev, data15_13tev_aod, mc15_13tev, mc15_13tev_aod

In [2]:
my_index = "data15_13tev_aod"

### Select a percentage of events to read. 
e.g. toRead=2 will read 2% of all events

In [3]:
toRead=2

In [4]:
my_query = {
    "min_score": 1-(toRead/100),
    "query" : {
        "function_score" : {
            "random_score" : { "seed":"123" }
        }
    }
}

### actually streaming events

In [5]:
t0 = time()

scroll = scan(es, query=my_query, index=my_index, scroll='5m', timeout="5m", size=100)

count = 0
for res in scroll:
    if not count%1000: print(count)
    count += 1
    
t1 = time()

print (count, 'events in %f seconds.' %(t1-t0), count/(t1-t0), "ev/s.")

0
522 events in 11.456021 seconds. 45.56556054810618 ev/s.
